# Using MIP to solve IBM's marketing optimization problem

Try to translate IBM's problem to the Google OR framework.

In [34]:
from ortools.linear_solver import pywraplp
import time
from __future__ import print_function

We have four product types:

  * car loan
  * savings
  * mortgage
  * pension
  
Each product has a different `productValue`: the revenue that can be obtained for the product on average. To get a fair representation of marketing across the various offers, each is allocated a `budgetShare`. 

In [35]:
products = ['Car loan', 'Savings', 'Mortgage', 'Pension']
productValue = [100, 200, 300, 400]
budgetShare = [0.6, 0.1, 0.2, 0.1]

  
Each product these can be offered over one of the following channels:

  * gift
  * newsletter
  * seminar
  
Each of these channels has different costs, and each has a different _influence factor_. We use the influence to weight the estimated value of the response accordingly.

In [36]:
channels = ['gift', 'newsletter', 'seminar']
cost = [20, 15, 23]
factor = [0.2, 0.05, 0.3]

Budget needs to be less than the available marketing budget of $ \$500$.

In [37]:
availableBudget = 500

Read in the offers data, originally from IBM and massaged. It gives the probability of taking an offer by each customer.

In [38]:
import pandas

product_probs = pandas.read_csv('offers_ibm_pivot.csv')
product_probs['Car loan'] = 0
# product_probs = product_probs.fillna(0)
product_probs = product_probs[['Car loan', 'Savings', 'Mortgage', 'Pension', 'customerid', 'name']]
product_probs.head()

,Car loan,Savings,Mortgage,Pension,customerid,name
0,0,0.28391,0.00000,0.80506,139068,Sandra J. Wynkoop
1,0,0.00000,0.10675,0.13221,139154,Roland Guérette
2,0,0.00000,0.10675,0.13221,139158,Fabien Mailhot
3,0,0.00000,0.10675,0.13221,139169,Christian Austerlitz
4,0,0.00000,0.10675,0.13221,139220,Steffen Meister


Instantiate the solver as an MIP problem.

In [39]:
solver = pywraplp.Solver('SolveCampaignProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Define the number of customers, the number of offers and the number of channel as $x_{ijk}$.

In [40]:
num_customers = product_probs.shape[0]
num_products = len(products)
num_channels = len(channels)

x = {}

for i in range(num_customers):
    for j in range(num_products):
        for k in range(num_channels):
            x[i, j, k] = solver.IntVar(0, 1, 'x[%i,%i,%i]' % (i, j, k))

### Set up the constraints

  1. Offer only one product per customer.
  2. Do not exceed the budget.
  3. Balance the offers/customers among products.
  

In [41]:
    ## offer only one product per customer
    for i in range(num_customers):
        solver.Add(solver.Sum([x[i, j, k] 
                               for j in range(num_products)
                               for k in range(num_channels)
                              ]) <= 1) 

    ## Do not exceed the budget
    solver.Add(solver.Sum([x[i, j, k]*cost[k]
                           for i in range(num_customers)
                           for j in range(num_products)
                           for k in range(num_channels)
                          ]) <= availableBudget)
    
    ## Balance the offers/customers among products
    for j in range(num_products):
        solver.Add(solver.Sum([x[i, j, k]
                               for i in range(num_customers)
                               for k in range(num_channels)
            ]) <= budgetShare[j]*solver.Sum([x[i, j, k]
                                            for i in range(num_customers)
                                            for j in range(num_products)
                                            for k in range(num_channels)
                                            ]) )
          

_There are a couple of constraints in the IBM version that don't seem to be real constraints; rather, they set up variables to calculate metrics of interest._

### Express the objective

We want to maximize revenue. Here $x_{ijk}$ denotes whether customer $i$ receives an offer for product $j$ over channel $k$, $f_k$ denotes the channel adjustment factor, $v_j$ the product value and $p_{ij}$ the probability that customer $i$ takes up product $j$.

$ \max R = \sum_{ijk} x_{ijk} \times f_k \times v_j \times p_{ij}$

In [42]:
#    solver.Minimize(solver.Sum([cost[i][j] * x[i, j] for i in range(num_workers)
#                                                     for j in range(num_tasks)]))

solver.Maximize(solver.Sum([x[i, j, k]*factor[k]*productValue[j]*product_probs.iloc[i, j]
                           for i in range(num_customers)
                           for j in range(num_products)
                           for k in range(num_channels)]))

### Invoke the solver

In [43]:
# Invoke the solver
sol = solver.Solve()

Print out the solution. We can print out more information about the constraints.

In [45]:
print('Total cost = %d' % (solver.Objective().Value()))
print()
for i in range(num_customers):
    for j in range(num_products):
        for k in range(num_channels):
            if x[i, j, k].solution_value() > 0:
                customerid = product_probs.loc[i, 'name']
                print('Customer %s assigned to product "%s" and channel "%s"' % (customerid, products[j], channels[k]))
    
print()
print('Time = ', solver.WallTime(), " milliseconds.")

Total cost = 423

Customer Sandra J. Wynkoop assigned to product "Car loan" and channel "newsletter"
Customer Roland Guérette assigned to product "Car loan" and channel "newsletter"
Customer Fabien Mailhot assigned to product "Car loan" and channel "newsletter"
Customer Christian Austerlitz assigned to product "Mortgage" and channel "seminar"
Customer Lee Tsou assigned to product "Car loan" and channel "newsletter"
Customer Sanaa' Hikmah Hakimi assigned to product "Car loan" and channel "newsletter"
Customer Miroslav Škaroupka assigned to product "Mortgage" and channel "seminar"
Customer George Blomqvist assigned to product "Car loan" and channel "newsletter"
Customer Will Henderson assigned to product "Pension" and channel "seminar"
Customer Yuina Ohira assigned to product "Mortgage" and channel "seminar"
Customer Vlad Alekseeva assigned to product "Car loan" and channel "newsletter"
Customer Cassio Lombardo assigned to product "Car loan" and channel "gift"
Customer Trinity Zelaya Mir